# Описание проекта


   Сеть фитнес-центров «Культурист-датасаентист» разрабатывает стратегию взаимодействия с клиентами на основе аналитических данных.
Распространённая проблема фитнес-клубов и других сервисов — отток клиентов. Чтобы понять, что клиент ушел, можно записать в отток тех, кто попросил закрыть договор или удалил аккаунт. Однако клиенты не всегда уходят демонстративно: чаще перестают пользоваться сервисом тихо.
   
   Индикаторы оттока зависят от специфики отрасли. Когда пользователь редко, но стабильно закупается в интернет-магазине — не похоже, что он «отвалился». А вот если две недели не заходит на канал с ежедневно обновляемым контентом, то возможно подписчик заскучал и решил уйти.

   Для фитнес-центра можно считать, что клиент попал в отток, если за последний месяц ни разу не посетил спортзал. Конечно, не исключено, что он уехал и по приезде продолжит ходить на фитнес. Однако чаще бывает наоборот. Если клиент начал новую жизнь с понедельника, немного походил в спортзал, а потом пропал — скорее всего, он не вернётся. Чтобы бороться с оттоком, отдел по работе с клиентами «Культуриста-датасаентиста» перевёл в электронный вид множество клиентских анкет.

Задача — провести анализ и подготовить план действий по удержанию клиентов.   
Для этого нужно:   
- спрогнозировать вероятность оттока (на уровне следующего месяца) для каждого клиента;
- сформировать типичные портреты клиентов: выделить несколько наиболее ярких групп и охарактеризовать их основные свойства;
- проанализировать основные признаки, наиболее сильно влияющие на отток;   
- сформулировать основные выводы и разработать рекомендации по повышению качества работы с клиентами:   
1) выделить целевые группы клиентов;    
2) предложить меры по снижению оттока;    
3) определить другие особенности взаимодействия с клиентами.   

# Описание данных


Данные клиента за предыдущий до проверки факта оттока месяц:   
- 'gender' — пол;   
- 'Near_Location' — проживание или работа в районе, где находится фитнес-центр;   
- 'Partner' — сотрудник компании-партнёра клуба (сотрудничество с компаниями, чьи сотрудники могут получать скидки на абонемент — в таком случае фитнес-центр хранит информацию о работодателе клиента);    
- Promo_friends — факт первоначальной записи в рамках акции «приведи друга» (использовал промо-код от знакомого при оплате первого абонемента);      
- 'Phone' — наличие контактного телефона;    
- 'Age' — возраст;     
- 'Lifetime' — время с момента первого обращения в фитнес-центр (в месяцах).    

Информация на основе журнала посещений, покупок и информация о текущем статусе абонемента клиента:    
- 'Contract_period' — длительность текущего действующего абонемента (месяц, 6 месяцев, год);   
- 'Month_to_end_contract' — срок до окончания текущего действующего абонемента (в месяцах);     
- 'Group_visits' — факт посещения групповых занятий;    
- 'Avg_class_frequency_total' — средняя частота посещений в неделю за все время с начала действия абонемента;    
- 'Avg_class_frequency_current_month' — средняя частота посещений в неделю за предыдущий месяц;   
- 'Avg_additional_charges_total' — суммарная выручка от других услуг фитнес-центра: кафе, спорттовары, косметический и массажный    салон.
- 'Churn' — факт оттока в текущем месяце. 

##  Изучение данных из файла

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import itertools
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
gym_churn = pd.read_csv('/Users/polina_lunyova/Documents/It/projects/final_version/8_Основы_машинного_обучения/gym_churn.csv')
display(gym_churn.head())
display(gym_churn.info())
display(gym_churn.describe())
print('Размер датафрейма:', gym_churn.shape)

In [ ]:
print('Количество дубликатов:', gym_churn.duplicated().sum())
display('Количество пропусков:', gym_churn.isna().sum())

В основном люди живут рядом с фитнес-центром.   
Половина клиентов пришли по партнерской программе, а 30% воспользовались промо-кодом от знакомого "приведи друга".  Практически у всех указан номер телефона. На груповые занятия ходят меньше половины клиентов(40%). Средняя длительность текущего действующего абонемента 4,7 месяцев, а средний срок до окончания текущего действующего абонемента 4,3 месяца. Среднее время с момента первого обращения в фитнес-центр 3,7 месяцев. Средний возраст клиентов 29-30 лет. Средняя суммарная выручка от других услуг фитнес-центра 147. Средняя частота посещений за все время с начала действия абонемента 1.87 раз в неделю. Средняя частота посещений за предыдущий месяц 1.76 раз в неделю. Отток клиентов на первый взгляд кажется не таким большим - 27%.


Дубликаты и пропуски не обнаружены.

##  Исследовательский анализ данных (EDA)

In [ ]:
# приведем названия столбцов к нижнему регистру 
gym_churn = gym_churn.rename(columns=str.lower)
gym_churn.head()

In [ ]:
# изменим тип данных
gym_churn['avg_additional_charges_total'] = gym_churn['avg_additional_charges_total'].astype('int64')
gym_churn['avg_class_frequency_total'] = gym_churn['avg_class_frequency_total'].astype('int64')
gym_churn['avg_class_frequency_current_month'] = gym_churn['avg_class_frequency_current_month'].astype('int64')
gym_churn.head()

### Cредние значения признаков в двух группах — тех, кто ушел в отток и тех, кто остался

В столбце 'Churn' (факт оттока в текущем месяце) считается, что:   
0- клинт остался;   
1- отток клиента.

In [ ]:
gym_churn.groupby('churn').agg('mean').T 

По данным таблицы можно сделать вывод, что:   
- распределение по гендеру и наличию телефона не отличается;    
- клиенты, живущие рядом и пришедшие по партнерской программе реже попадают в отток;    
- клиенты, пришедшие по совету друзей (акция приведи друга) в два раза меньше попадают в отток;    
- у клиентов, с абонементом больше одного месяца вероятность попасть в отток меньше;    
- у клиентов, которые посещяют групповые тренировки, вероятность попасть в отток в два раза меньше;    
- более молодые клиенты подвержены оттоку;    
- у оставшихся клиентов суммарная выручка от других услуг фитнес-центра немного больше;    
- у отсавшихся клиентов срок окончания действия абонемента значительно выше; 
- новые клиенты, занимающиеся только первый месяц, наиболее подвержены оттоку;
- клиенты, с небольшим средним посещением в неделю за прошлый месяц и все время подвержены оттоку.     

Следовательно новые клинеты, которые далеко живут, пришли не по партнерском программе или акции, берущие абонемент на месяц и редко посещают фитнес-центр наиболее подвержены оттоку, что в принципе и логично.

In [ ]:
mean_avg_class = gym_churn.groupby(['avg_class_frequency_total', 'avg_class_frequency_current_month'])['churn'].mean()\
.reset_index().sort_values(by='churn', ascending=False).round(2)
mean_avg_class

Клиенты, у которых средняя частота посещений в неделю за предыдущий месяц варьируется от 0 до 2 посещений, наиболее подвержены к оттоку(уход из фитнес-центра).

### Cтолбчатые гистограммы и распределения признаков для тех, кто ушёл (отток) и тех, кто остался (не попали в отток)

In [ ]:
# строим гистограмму признаков клиентов
sns.set(style="darkgrid")
fig, axs = plt.subplots(3, 3, figsize=(15, 15))

sns.histplot(data=gym_churn, x="gender", kde=True, color="skyblue", ax=axs[0, 0])
sns.histplot(data=gym_churn, x="near_location", kde=True, color="olive", ax=axs[0, 1])
sns.histplot(data=gym_churn, x="partner", kde=True, color="gold", ax=axs[1, 0])
sns.histplot(data=gym_churn, x="promo_friends", kde=True, color="teal", ax=axs[1, 1])
sns.histplot(data=gym_churn, x="phone", kde=True, color="red", ax=axs[0, 2])
sns.histplot(data=gym_churn, x="contract_period", kde=True, color="blue", ax=axs[1, 2])
sns.histplot(data=gym_churn, x="group_visits", kde=True, color="green", ax=axs[2, 2])
sns.histplot(data=gym_churn, x="age", kde=True, color="grey", ax=axs[2, 1])
sns.histplot(data=gym_churn, x="avg_additional_charges_total", kde=True, color="pink", ax=axs[2, 0])
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 12))

sns.histplot(data=gym_churn, x="month_to_end_contract", kde=True, color="orange", ax=axs[0, 0])
sns.histplot(data=gym_churn, x="lifetime", kde=True, color="yellow", ax=axs[0, 1])
sns.histplot(data=gym_churn, x="avg_class_frequency_total", kde=True, color="grey", ax=axs[1, 0])
sns.histplot(data=gym_churn, x="avg_class_frequency_current_month", kde=True, color="pink", ax=axs[1, 1])

plt.show()

### Матрица корреляций

In [ ]:
corr_matrix = gym_churn.corr()
plt.figure(figsize=(15, 10))
sns.heatmap(corr_matrix , annot = True, square=True, fmt = '0.2f', linewidths=.3, cmap="YlGnBu") 
plt.show()

Мультиколлинеарность возникает, когда есть группа линейно зависимых признаков — взаимозависимых или очень сильно скоррелированных. Если коэффициент корреляции между двумя признаками слишком большой (часто больше 0.8), с линейной регрессией возникнуть проблемы. А именно — нерепрезентативные веса и переобученная под эти признаки модель.

Среди имеющихся признаков выявлены мультиколлинеарные, такие как contract_period(длительность текущего действующего абонемента) и month_to_end_contract(срок до окончания абонемента), avg_class_frequency_current_month(средняя частота посещений в неделю за предыдущий месяц) и avg_class_frequency_total(средняя частота посещений в неделю за все время с начала действия абонемента).   

Значительно влияющие на отток признаки promo_friends(акция "приведи друга") и partner(партнерская программа), contract_period(длительность абонемента) и partner(партнерская программа), month_to_end_contract(срок до окончания абонемента) и partner(партнерская программа).   

Также имеются множества признаков, которые не влияют(или практически не влияют) на отток такие как phone и gender, phone и avg_class_frequency_current_month и т.д.

In [ ]:
# удалим мультиколлинеарные (сильно коррелирующие) переменные
gym_churn = gym_churn.drop('month_to_end_contract', axis = 1) # убираем один из пары коррелирующих признаков
gym_churn = gym_churn.drop('avg_class_frequency_total', axis = 1)

## Построение модели прогнозирования оттока клиентов


**Построим модель бинарной классификации клиентов, где целевой признак — факт оттока клиента в следующем месяце**

In [ ]:
# разделим наши данные на признаки (матрица X) и целевую переменную (y)
X = gym_churn.drop('churn', axis = 1)
y = gym_churn['churn']

# разделим модель на обучающую и валидационную выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

# обучим StandartScaler на обучающей выборке
scaler = StandardScaler() # создаём объект класса scaler
scaler.fit(X_train) # обучаем стандартизатор

# преобразуем обучающий и валидационные наборы данных
X_train_st = scaler.transform(X_train) 
X_test_st = scaler.transform(X_test)

**Обучим модель на train-выборке логистической регрессией и случайным лесом**


**Логистическая регрессия** - это алгоритм для решения задачи бинарной классификации, используемая для прогнозирования вероятности возникновения некоторого события путём его сравнения с логистической кривой.    
   
Модели, обученные алгоритмами для бинарной классификации, могут не просто прогнозировать финальное значение класса для какого-то объекта или клиента, а ещё и оценивать вероятность рассматриваемого события.   

Метрика accuracy работает не всегда, а только при условии баланса классов — когда объектов каждого класса примерно поровну, 50% : 50%(сбалансированные)

Чтобы оценить модель без привязки к соотношению классов, рассчитывают метрики:
точность (precision) и полнота (recall)   
precision говорит, какая доля прогнозов относительно "1" класса верна. То есть смотрим долю правильных ответов только среди целевого класса.
Вторая метрика нацелена на минимизацию противоположных рисков — recall показывает, сколько реальных объектов "1" класса вы смогли обнаружить с помощью модели.

Существует еще один тип ансамбля моделей (модель машинного обучения) — **случайный лес**.  Он генерирует множество различных независимых друг от друга деревьев разными способами (берёт разные подвыборки, разные признаки) и на основании их ответов формирует итоговое решение. Алгоритм случайного леса усредняет ответы всех деревьев (в задаче регрессии) или выбирает голосованием (в классификации) тот ответ, который  большинство деревьев в лесу считает правильным.   

**F1 оценка** - это классификация ошибки метрики, которая как и любая другая ошибка, помогает нам оценить производительность алгоритма и включает в себя основные параметры точности и полноты.

In [ ]:
# создадим функцию для расчетов метрик
def metrics(name, mdl):
    mdl.fit(X_train_st, y_train)
    y_pred = mdl.predict(X_test_st)
    print("Модель прогнозирования -", name)
    acc = accuracy_score(y_test, y_pred)
    print("Доля правильных ответов:", acc.round(2))
    precision = precision_score(y_test, y_pred).round(2)
    print("Точность:", precision)
    recall = recall_score(y_test, y_pred).round(2)
    print("Полнота:", recall)
    f1 = f1_score(y_test, y_pred).round(2)
    print("F1:", f1, '\n')

model = LogisticRegression(random_state=0)   
metrics('Логистическая регрессия', model)

model = RandomForestClassifier(random_state=0)
parametrs = { 'n_estimators': range (10, 51, 10),
              'max_depth': range (1,13, 2)}
grid = GridSearchCV(model, parametrs, cv=5)
grid.fit(X_train, y_train)
metrics('Случайный лес', model)

Обе модели показали хороший результат (около 90% правильных ответов).

##  Кластеризация клиентов

Кластеризация — задача группировки множества объектов на подмножества (кластеры) таким образом, чтобы объекты из одного кластера были более похожи друг на друга, чем на объекты из других кластеров по какому-либо критерию.

In [ ]:
# обязательная стандартизация данных перед работой с алгоритмами
sc = StandardScaler()
X_sc = sc.fit_transform(X)

linked = linkage(X_sc, method = 'ward')

wcss = []
for i in range(1, 11):
    km = KMeans(n_clusters=i, init="k-means++", random_state=0)
    labels = km.fit_predict(X_sc) 
    wcss.append(km.inertia_)

plt.figure(figsize=(15, 10))  
dendrogram(linked, orientation='top')
plt.title('Иерархическая кластеризация')
plt.show()

# сохраняем метки кластера в поле нашего датасета
gym_churn['cluster_km'] = labels

In [ ]:
count_km = gym_churn['cluster_km'].nunique()
print("Количество кластеров:", count_km)

**Посмотрим на средние значения признаков для кластеров**

In [ ]:
# выводим статистику по средним значениям наших признаков по кластеру
clusters = gym_churn.groupby(['cluster_km']).mean()
clusters

При рассмотрении средних значений признаков для кластеров, можно заметить, что: 
- самый большой отток клиентов идет в 3 кластере, а самый маленький в первом;
- самое большое расстояние от фитнес-центра в 0 кластере, а самое короткое во втором и первом;
- наименьшее число клиентов, пришедшее по промокоду "пригласи друга" в 0 кластере;
- наибольшее число клиентов, пришедшее по партнерской программе в кластере 1;
- наиболее высокая длительность абонемента в кластере 1, а наименьшая в кластере 3;
- в кластере 0 самое маленькое количество посещений групповых занятий, в кластере 1 - наибольшее;
- наибольшая суммарная выручка от других услуг фитнес-центра в кластере 1, а наименьшая в кластерах 0 и 3;
- наибольший срок до окончания текущего действующего абонемента в кластере 1, наименьший в кластере 3;
- наибольшее время с момента первого обращения в фитнес-центр в кластерах 1 и 4, наименьшее в 3 и 0;
- наименьшая средняя частота посещений в неделю за все время и за предыдущий месяц с начала действия абонемента у кластера 3, наибольшая у кластера 4.


Для лучшего понимания характеристик кластеров разобьем их на группы:    
   
**0 Кластер** - самое большое расстояние от фитнес-центра, наименьшее число людей, пришедшее по промокоду "пригласи друга", наименьшее количество посещений групповых занятий, наименьшая суммарная выручка от других услуг фитнес-центра, наличие небольшого времени с момента первого обращения в фитнес-центр.
   
**1 Кластер** - самый маленький отток клиентов, наибольшее число клиентов, пришедшее по партнерской программе, небольшое расстояние до фтнес-центра, высокая длительность абонемента, наибольшая суммарная выручка от других услуг фитнес-центра, наибольшее количество посещений групповых занятий, наибольший срок до окончания текущего действующего абонемента, наибольшее время с момента первого обращения в фитнес-центр.
    
**2 Кластер** - средние, не выбивающие показатели признаков, но клиенты не оставляют номера телефона. 
    
**3 Кластер** - самый большой отток клиентов, наименьшая длительность абонемента, небольшая суммарная выручка от других услуг фитнес-центра, наличие небольшого времени с момента первого обращения в фитнес-центр, наименьшая средняя частота посещений в неделю за все время и за предыдущий месяц с начала действия абонемента.
    
**4 Кластер** - наибольшее время с момента первого обращения в фитнес-центр, а также наибольшая средняя частота посещений в неделю за все время и за предыдущий месяц с начала действия абонемента.     



**Построим распределения признаков для кластеров**

In [ ]:
for value in [0, 1, 2, 3, 4]:
    gym_churn[gym_churn['cluster_km']==value].hist(figsize=(18, 12))
    plt.suptitle('Распределение признаков для кластера {}'.format(value), fontsize = 15)
    plt.show()

**Посчитаем долю оттока для каждого полученного кластера**

In [ ]:
# посмотрим средние значения оттока в кластерах
gym_churn.groupby('cluster_km').agg({'churn':'mean'}).sort_values(by='churn')

Cредние значения оттока в кластерах не отличаются. Наиболее склонен к оттоку кластер 3, а наименее кластер 1.

## Выводы и базовые рекомендации по работе с клиентами

В ходе исследования мы сопоставили признаки клиентов с оттоком(отток клиентов составил 27%), благодаря которому узнали какие признаки подвержены наибольшему оттоку.   Далее построили модели прогнозирования оттока клиентов, где целевой признак — факт оттока клиента в следующем месяце. Обе модели показали хороший результат (около 90% правильных ответов).

Далее мы разделили клиентов на кластеры (группы):   

**0 Кластер** - самое большое расстояние от фитнес-центра, наименьшее число людей, пришедшее по промокоду "пригласи друга", наименьшее количество посещений групповых занятий, наименьшая суммарная выручка от других услуг фитнес-центра, наличие небольшого времени с момента первого обращения в фитнес-центр.
   
**1 Кластер** - самый маленький отток клиентов, наибольшее число клиентов, пришедшее по партнерской программе, небольшое расстояние до фтнес-центра, высокая длительность абонемента, наибольшая суммарная выручка от других услуг фитнес-центра, наибольшее количество посещений групповых занятий, наибольший срок до окончания текущего действующего абонемента, наибольшее время с момента первого обращения в фитнес-центр.
    
**2 Кластер** - средние, не выбивающие показатели признаков, но клиенты не оставляют номера телефона. 
    
**3 Кластер** - самый большой отток клиентов, наименьшая длительность абонемента, небольшая суммарная выручка от других услуг фитнес-центра, наличие небольшого времени с момента первого обращения в фитнес-центр, наименьшая средняя частота посещений в неделю за все время и за предыдущий месяц с начала действия абонемента.
    
**4 Кластер** - наибольшее время с момента первого обращения в фитнес-центр, а также наибольшая средняя частота посещений в неделю за все время и за предыдущий месяц с начала действия абонемента.      

Кластер 1 подвержен наименьшему оттоку клиентов, наибольшему - кластеры 0 и 3.
          
По результатам исследования можно сделать вывод, что в основном клиент уходит, когда живет на большом расстоянии от фитнес-центра, не использует никаких бонусных программ или акций, берет абонеменмент на месяц, чтобы протестировать его.

Рекомендации по уменьшению оттока клиентов:
- предложить клиенту выгодное предложение в виде скидки при покупки абонемента от полугода;
- если клиент на протяжении месяца не появлялся в фитнес-центре(или был 1-2 раза), позвонить/написать и замотивировать сходить на тренировку(к примеру предложить бесплатное занятие с фитнес инструктором);
- в случае ухода клиента узнать почему он принял такое решение и попытаться удержать его;
- иногда рассылать ушедшим клиентам заманчивые предложения в виде акций, промокодов и тд.